In [1]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from plotting_utils import *
import torch

In [3]:
import guided_diffusion_v3 as gd

def create_gaussian_diffusion(
    *,
    steps=1000,
    learn_sigma=False,
    sigma_small=False,
    noise_schedule="linear",
    use_kl=False,
    predict_xstart=False,
    rescale_timesteps=False,
    rescale_learned_sigmas=False,
    timestep_respacing="",
):
    betas = gd.get_named_beta_schedule(noise_schedule, steps)
    if use_kl:
        loss_type = gd.LossType.RESCALED_KL
    elif rescale_learned_sigmas:
        loss_type = gd.LossType.RESCALED_MSE
    else:
        loss_type = gd.LossType.MSE
    if not timestep_respacing:
        timestep_respacing = [steps]
    return gd.SpacedDiffusion(
        use_timesteps=gd.space_timesteps(steps, timestep_respacing),
        betas=betas,
        model_mean_type=(
            gd.ModelMeanType.EPSILON if not predict_xstart else gd.ModelMeanType.START_X
        ),
        model_var_type=(
            (
                gd.ModelVarType.FIXED_LARGE
                if not sigma_small
                else gd.ModelVarType.FIXED_SMALL
            )
            if not learn_sigma
            else gd.ModelVarType.LEARNED_RANGE
        ),
        loss_type=loss_type,
        rescale_timesteps=rescale_timesteps,
    )

diffusion = create_gaussian_diffusion()

In [5]:
# input_folder = r'C:\Users\u_imagenes\Documents\smerino\testing\input'
# output_folder = r'C:\Users\u_imagenes\Documents\smerino\testing\target_enh'
input_folder = r'C:\Users\sebas\Documents\Data\DiffusionBeamformer\input_overfit'
output_folder = r'C:\Users\sebas\Documents\Data\DiffusionBeamformer\target_overfit'

data = gd.CustomDataset(input_folder, output_folder, transform=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device('cpu'))


In [6]:
from model6 import UNETv12

device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device('cpu'))
save_dir = r'.\weights\v12'
training_epochs = 5
model = UNETv12(rrdb_blocks=1).to(device)
model.load_state_dict(torch.load(f"{save_dir}\\model_{training_epochs}.pth", map_location=device))
print("Num params: ", sum(p.numel() for p in model.parameters()))

FileNotFoundError: [Errno 2] No such file or directory: '.\\weights\\v12\\model_5.pth'